In [49]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [25]:
questions = [
    'Best book?', 'How old?', 'Your job?', 'Best food?', 'Pets?',
    'Coffee or tea?', 'Best movie?', 'Hobbies?', 'Favorite color?', 'Morning person?',
    'Travel dream?', 'Workout?', 'Dream job?', 'Your hero?', 'Best book?',
    'Pets?', 'Morning person?', 'Workout?', 'Coffee or tea?', 'Dream job?',
    'Best food?', 'Travel dream?', 'Your job?', 'Best movie?', 'Favorite color?',
    'How old?', 'Hobbies?', 'Your hero?', 'Pets?', 'Coffee or tea?',
    'Morning person?', 'Best movie?', 'Workout?', 'Dream job?', 'Your job?',
    'Favorite color?', 'Travel dream?', 'Best food?', 'How old?', 'Hobbies?',
    'Your hero?', 'Best book?', 'Pets?', 'Coffee or tea?', 'Morning person?',
    'Best movie?', 'Workout?', 'Dream job?', 'Your job?', 'Favorite color?'
]

answers = [
    'Sushi is the best food ever.', "The best movie is 'Inception'.", 'My hero is Marie Curie.', 'I have two dogs.', "My favorite book is '1984' by George Orwell.",
    'Definitely coffee.', 'No, I\'m definitely not a morning person.', 'Yes, I workout 5 days a week.', 'I dream of being an astronaut.', 'I\'m 29 years old.',
    'I love the color blue.', 'I enjoy hiking and painting.', 'I dream of traveling to Japan.', 'I\'m a software engineer.', 'Not much, you?',
    'My favorite book is \'1984\' by George Orwell.', 'I have two dogs.', 'Definitely coffee.', 'Yes, I workout 5 days a week.', 'I dream of being an astronaut.',
    'Sushi is the best food ever.', 'I dream of traveling to Japan.', 'I\'m a software engineer.', "The best movie is 'Inception'.", 'I love the color blue.',
    'I\'m 29 years old.', 'I enjoy hiking and painting.', 'My hero is Marie Curie.', 'Not much, you?', 'Definitely coffee.',
    'No, I\'m definitely not a morning person.', "The best movie is 'Inception'.", 'Yes, I workout 5 days a week.', 'I dream of being an astronaut.', 'I\'m a software engineer.',
    'I love the color blue.', 'I dream of traveling to Japan.', 'Sushi is the best food ever.', 'I\'m 29 years old.', 'I enjoy hiking and painting.',
    'My hero is Marie Curie.', 'Not much, you?', 'My favorite book is \'1984\' by George Orwell.', 'Definitely coffee.', 'No, I\'m definitely not a morning person.',
    "The best movie is 'Inception'.", 'Yes, I workout 5 days a week.', 'I dream of being an astronaut.', 'I\'m a software engineer.', 'I love the color blue.'
]


In [26]:
###############################
#        max_len = 13         #
###############################

sorted_ques = []
sorted_ans = []
for i in range(len(questions)):
    if len(questions[i]) < 13:
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])



###############################
#                             #
###############################
len(sorted_ques)

35

In [27]:
import re
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

In [28]:
clean_ques = []
clean_ans = []

for line in sorted_ques:
    clean_ques.append(clean_text(line))
        
for line in sorted_ans:
    clean_ans.append(clean_text(line))



## delete
del(answers, questions)

In [29]:
###############################
#                             #
###############################


for i in range(len(clean_ans)):
    clean_ans[i] = ' '.join(clean_ans[i].split()[:11])



###############################
#                             #
###############################

del(sorted_ans, sorted_ques)

In [30]:
## trimming
clean_ans=clean_ans[:30000]
clean_ques=clean_ques[:30000]
## delete

In [31]:
###  count occurences ###
word2count = {}

for line in clean_ques:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in clean_ans:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1


In [32]:
###  remove less frequent ###
thresh = 0 #change it later

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1

In [33]:
for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'



tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1
    
    

#vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

In [34]:
### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}


In [35]:
encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

In [36]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 13, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 13, padding='post', truncating='post')




decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 13, padding='post', truncating='post')

In [37]:
# decoder_final_output, decoder_final_input, encoder_final, vocab, inv_vocab

VOCAB_SIZE = len(vocab)
MAX_LEN = 13

print(decoder_final_output.shape, decoder_inp.shape, encoder_inp.shape, len(vocab), len(inv_vocab), inv_vocab[0])

(35, 13) (35, 13) (35, 13) 63 62 <PAD>


In [38]:
vocab

{'best': 0,
 'book': 1,
 'how': 2,
 'old': 3,
 'your': 4,
 'job': 5,
 'food': 6,
 'pets': 7,
 'movie': 8,
 'hobbies': 9,
 'workout': 10,
 'dream': 11,
 'hero': 12,
 'sushi': 13,
 'is': 14,
 'the': 15,
 'ever': 16,
 'inception': 17,
 'my': 18,
 'marie': 19,
 'curie': 20,
 'i': 21,
 'have': 22,
 'two': 23,
 'dogs': 24,
 'favorite': 25,
 '1984': 26,
 'by': 27,
 'george': 28,
 'orwell': 29,
 'no': 30,
 'am': 31,
 'definitely': 32,
 'not': 33,
 'a': 34,
 'morning': 35,
 'person': 36,
 'yes': 37,
 '5': 38,
 'days': 39,
 'week': 40,
 'enjoy': 41,
 'hiking': 42,
 'and': 43,
 'painting': 44,
 'of': 45,
 'traveling': 46,
 'to': 47,
 'japan': 48,
 'software': 49,
 'engineer': 50,
 'much': 51,
 'you': 52,
 'coffee': 53,
 'being': 54,
 'an': 55,
 'astronaut': 56,
 '29': 57,
 'years': 58,
 '<PAD>': 0,
 '<EOS>': 60,
 '<OUT>': 61,
 '<SOS>': 62}

In [39]:
inv_vocab

{0: '<PAD>',
 1: 'book',
 2: 'how',
 3: 'old',
 4: 'your',
 5: 'job',
 6: 'food',
 7: 'pets',
 8: 'movie',
 9: 'hobbies',
 10: 'workout',
 11: 'dream',
 12: 'hero',
 13: 'sushi',
 14: 'is',
 15: 'the',
 16: 'ever',
 17: 'inception',
 18: 'my',
 19: 'marie',
 20: 'curie',
 21: 'i',
 22: 'have',
 23: 'two',
 24: 'dogs',
 25: 'favorite',
 26: '1984',
 27: 'by',
 28: 'george',
 29: 'orwell',
 30: 'no',
 31: 'am',
 32: 'definitely',
 33: 'not',
 34: 'a',
 35: 'morning',
 36: 'person',
 37: 'yes',
 38: '5',
 39: 'days',
 40: 'week',
 41: 'enjoy',
 42: 'hiking',
 43: 'and',
 44: 'painting',
 45: 'of',
 46: 'traveling',
 47: 'to',
 48: 'japan',
 49: 'software',
 50: 'engineer',
 51: 'much',
 52: 'you',
 53: 'coffee',
 54: 'being',
 55: 'an',
 56: 'astronaut',
 57: '29',
 58: 'years',
 60: '<EOS>',
 61: '<OUT>',
 62: '<SOS>'}

In [40]:
from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

decoder_final_output.shape

(35, 13, 63)

In [43]:
embeddings_index = {}
with open('../dataset/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("GloVe Loded!")

GloVe Loded!


In [44]:
embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index)+1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix = embedding_matrix_creater(50, word_index=vocab) 

In [45]:
embedding_matrix.shape

(64, 50)

In [46]:
embedding_matrix[0]

array([-9.15719986e-01,  6.03450000e-01, -3.10770005e-01,  2.84330010e-01,
        5.46100020e-01, -3.92290018e-03, -9.46399987e-01, -3.02100003e-01,
        7.11499974e-02,  8.23849976e-01, -1.69489995e-01,  4.10540015e-01,
       -4.86220002e-01,  6.14830017e-01,  7.04680026e-01, -6.00319982e-01,
        8.93819988e-01,  1.17810003e-01, -7.78310001e-01, -5.22059977e-01,
       -1.00769997e-01,  3.83920014e-01,  2.78919995e-01,  3.72020006e-01,
        4.93739992e-01, -1.14950001e+00, -1.15050006e+00, -7.20889986e-01,
       -3.80900018e-02, -4.56990004e-01,  3.34249997e+00,  5.66600025e-01,
        8.89440009e-04, -2.00299993e-01,  5.36620021e-01,  2.84779996e-01,
       -8.38219970e-02,  9.75350022e-01, -3.59849989e-01, -1.08179998e+00,
       -5.15550002e-02,  3.04529995e-01,  4.71549993e-03, -2.52139986e-01,
       -3.51509988e-01,  2.61350006e-01,  1.99980006e-01, -5.60319982e-02,
        7.49880001e-02,  7.60919988e-01])

In [47]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Bidirectional, Concatenate, Dropout, Attention

embed = Embedding(VOCAB_SIZE+1, 
                  50, 
                  
                  input_length=13,
                  trainable=True)

embed.build((None,))
embed.set_weights([embedding_matrix])

Metal device set to: Apple M1


2024-03-27 11:33:14.367221: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-27 11:33:14.367856: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [50]:
#building model
enc_inp = Input(shape=(13, ))

#embed = Embedding(VOCAB_SIZE+1, 50, mask_zero=True, input_length=13)(enc_inp)
enc_embed = embed(enc_inp)
enc_lstm = Bidirectional(LSTM(400, return_state=True, dropout=0.05, return_sequences = True))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = enc_lstm(enc_embed)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

enc_states = [state_h, state_c]


dec_inp = Input(shape=(13, ))
dec_embed = embed(dec_inp)
dec_lstm = LSTM(400*2, return_state=True, return_sequences=True, dropout=0.05)
output, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

# attention
attn_layer = AttentionLayer()
attn_op, attn_state = attn_layer([encoder_outputs, output])
decoder_concat_input = Concatenate(axis=-1)([output, attn_op])


dec_dense = Dense(VOCAB_SIZE, activation='softmax')
final_output = dec_dense(decoder_concat_input)

model = Model([enc_inp, dec_inp], final_output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 13)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 13)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 13, 50)       3200        ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 bidirectional_1 (Bidirectional  [(None, 13, 800),   1443200     ['embedding[2][0]']          

In [51]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [52]:
model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=500, batch_size=16, validation_split=0.1)

Epoch 1/500


2024-03-27 11:55:33.637719: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-27 11:55:35.130830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:35.583154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:35.595333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:35.966030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:37.149307: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:37.551926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114

2/2 [==============================] - ETA: 0s - loss: 3.5625 - acc: 0.2605    

2024-03-27 11:55:39.566718: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:39.690470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:39.699195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 11:55:39.908138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 7s 2s/step - loss: 3.5625 - acc: 0.2605 - val_loss: 2.5836 - val_acc: 0.5000
Epoch 2/500
2/2 [==============================] - 1s 377ms/step - loss: 2.3661 - acc: 0.5236 - val_loss: 2.0927 - val_acc: 0.5000
Epoch 3/500
2/2 [==============================] - 1s 369ms/step - loss: 2.0219 - acc: 0.5285 - val_loss: 1.9968 - val_acc: 0.5000
Epoch 4/500
2/2 [==============================] - 1s 435ms/step - loss: 1.9136 - acc: 0.5285 - val_loss: 1.9191 - val_acc: 0.5385
Epoch 5/500
2/2 [==============================] - 1s 396ms/step - loss: 1.8554 - acc: 0.5509 - val_loss: 1.8646 - val_acc: 0.5385
Epoch 6/500
2/2 [==============================] - 1s 352ms/step - loss: 1.7868 - acc: 0.5608 - val_loss: 1.7331 - val_acc: 0.5577
Epoch 7/500
2/2 [==============================] - 1s 354ms/step - loss: 1.7163 - acc: 0.5732 - val_loss: 1.6819 - val_acc: 0.5769
Epoch 8/500
2/2 [==============================] - 1s 349ms/step - loss: 1.6561 - acc: 0.5881 - va

In [53]:
model.save('chatbot.h5')
model.save_weights('chatbot_weights.h5')

In [54]:
#inference
enc_model = tf.keras.models.Model(enc_inp, [encoder_outputs, enc_states])

In [58]:
decoder_state_input_h = tf.keras.layers.Input(shape=( 400 * 2,))
decoder_state_input_c = tf.keras.layers.Input(shape=( 400 * 2,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]

#decoder_output = dec_dense(decoder_outputs)

dec_model = Model([dec_inp, decoder_states_inputs],
                                      [decoder_outputs] + decoder_states)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

print("##########################################")
print("#       start chatting version. 1.0      #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    
    prepro1 = input("you : ")
    try:
        prepro1 = clean_text(prepro1)
        prepro = [prepro1]
        
        txt = []
        for x in prepro:
            lst = []
            for y in x.split():
                try:
                    lst.append(vocab[y])
                except:
                    lst.append(vocab['<OUT>'])
            txt.append(lst)
        txt = pad_sequences(txt, 13, padding='post')


        ###
        enc_op, stat = enc_model.predict( txt )

        empty_target_seq = np.zeros( ( 1 , 1) )
        empty_target_seq[0, 0] = vocab['<SOS>']
        stop_condition = False
        decoded_translation = ''


        while not stop_condition :

            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + stat )

            ###
            ###########################
            attn_op, attn_state = attn_layer([enc_op, dec_outputs])
            decoder_concat_input = Concatenate(axis=-1)([dec_outputs, attn_op])
            decoder_concat_input = dec_dense(decoder_concat_input)
            ###########################

            sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )

            sampled_word = inv_vocab[sampled_word_index] + ' '

            if sampled_word != '<EOS> ':
                decoded_translation += sampled_word           


            if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
                stop_condition = True

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            stat = [ h , c ] 

        print("chatbot attention : ", decoded_translation )
        print("==============================================")

    except:
        print("sorry didn't got you , please type again :( ")

##########################################
#       start chatting version. 1.0      #
##########################################


you :  best food


1/1 [==============================] - 0s 13ms/step


2024-03-27 12:35:24.016092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-27 12:35:24.048843: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 11ms/step
chatbot attention :  sushi is the <PAD> food ever 


you :  which colour do you like ?


1/1 [==============================] - 0s 10ms/step
chatbot attention :  i am 29 years old 
